In [1]:
import pandas as pd

In [2]:
excel_file_path = 'ValidationReport.xlsx'
excel_data = pd.read_excel(excel_file_path, sheet_name=None)
type(excel_data)


dict

In [3]:
validation_data = excel_data["Validation"]
validation_data

,SourceSchema,SourceTable
0,AustinSchema,AustinTable
1,DallasSchema,DallasTable
2,HoustonSchema,HoustonTable
3,SanJoseSchema,SanJoseTable


In [4]:
table_name = "SanJoseTable"
source_schema = validation_data[validation_data['SourceTable'] == table_name]
source_schema


,SourceSchema,SourceTable
3,SanJoseSchema,SanJoseTable


In [5]:
source_schema["SourceSchema"]

3    SanJoseSchema
Name: SourceSchema, dtype: object

In [6]:
source_schema["SourceSchema"].values

array(['SanJoseSchema'], dtype=object)

In [7]:
source_schema["SourceSchema"].values[0]

'SanJoseSchema'

In [23]:
def generate_select_statement(excel_file_path: str, table_name: str) -> str:
    """
    Generate a SELECT statement for a given table name from the Excel file.

    Parameters:
    excel_file_path (str): Path to the Excel file (.xlsx) containing the data.
    table_name (str): Name of the table for which to generate the SELECT statement.

    Returns:
    str: The generated SELECT statement.
    """
    # Load the Excel data
    excel_data = pd.read_excel(excel_file_path, sheet_name=None)

    # Extract SourceSchema for the given table from the 'Validation' sheet
    validation_data = excel_data['Validation']
    source_schema = validation_data[validation_data['SourceTable'] == table_name]['SourceSchema'].values

    if not source_schema:
        return f"Table name '{table_name}' not found in the validation data."
    source_schema = source_schema[0]
    print(f"SourceSchema is {source_schema}.")

    # Generate the SELECT * statement using SourceSchema and the given table name
    select_statement = f"SELECT * FROM {source_schema}.{table_name} WHERE "

    # Generate the WHERE clause using all columns for the specified table
    data = excel_data[table_name].iloc[1:, :]  # Skip the header row
    where_conditions = []

    for _, row in data.iterrows():
        conditions = []
        for column, value in row.items():
            if isinstance(value, str):
                condition = f"{column} = '{value}'"
            else:
                condition = f"{column} = {value}"
            conditions.append(condition)
        where_conditions.append(" AND ".join(conditions))

    select_statement += ' OR '.join(f'({condition})' for condition in where_conditions)
    return select_statement


In [24]:
# Example usage
excel_file_path = "ValidationReport.xlsx"
table_name = "AustinTable"
select_query = generate_select_statement(excel_file_path, table_name)
print("Generated SELECT statement:")
print(select_query)

SourceSchema is AustinSchema.
Generated SELECT statement:
SELECT * FROM AustinSchema.AustinTable WHERE (Name = 'AustinBob' AND Age = 25 AND City = 'Dallas') OR (Name = 'AustinCarol' AND Age = 35 AND City = 'Houston') OR (Name = 'AustinDavid' AND Age = 40 AND City = 'San Jose') OR (Name = 'AustinEve' AND Age = 28 AND City = 'Austin')
